<h2> Model training </h2>

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [12]:
## UČITAVANJE TRAIN I TEST SKUPA

X_train = pd.read_hdf('X_train.h5' , key = 'X_train')
X_test = pd.read_hdf('X_test.h5', key = 'X_test')
y_train = X_train['PRIJEVREMENI_RASKID'].ravel()
X_train.drop(['PRIJEVREMENI_RASKID'], axis = 1, inplace = True)

## to je normalizirani skup, spreman za treniranje

In [16]:
### s odabirom znacajki
#X_train = pd.read_hdf('X_train_ft.h5' , key = 'X_train_ft')
#X_test = pd.read_hdf('X_test_ft.h5', key = 'X_test_ft')
#y_train = X_train['PRIJEVREMENI_RASKID'].ravel()
#X_train.drop(['PRIJEVREMENI_RASKID'], axis = 1, inplace = True)


<h3> XGBoost </h3>

In [3]:
!pip install xgboost

In [13]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [5]:
# https://www.kaggle.com/babatee/intro-xgboost-classification
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

In [7]:
train_model1 = model1.fit(X_train, y_train)
train_model2 = model2.fit(X_train, y_train)

In [8]:
#from sklearn.metrics import classification_report

pred1 = train_model1.predict(X_test)
pred2 = train_model2.predict(X_test)

In [10]:
# ISPIS RJESENJA

y_test = pred2
y_test = np.where(y_test==0,'N','Y')
student = pd.read_excel('eval_dataset_nan.xlsx')
student['PRIJEVREMENI_RASKID'] = y_test
student.to_csv('student.csv')

<h4> Hyperparameter tuning </h4>

In [ ]:
model1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [ ]:
train_model1 = model1.fit(X_train, y_train)
pred3 = train_model1.predict(X_test)
# print("Accuracy for model 3: %.2f" % (accuracy_score(y_test, pred3) * 100))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
#print("Accuracy for model 4: %.2f" % (accuracy_score(y_test, pred4) * 100))

In [ ]:
gsearch.best_params_ # pokaze koji je najbolji parametar odabran

In [ ]:
##ISPIS RJESENJA
y_test = pred4
y_test = np.where(y_test==0,'N','Y')
student = pd.read_excel('eval_dataset_nan.xlsx')
student['PRIJEVREMENI_RASKID'] = y_test
student.to_csv('student.csv')

In [ ]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model5 = gsearch2b.fit(X_train, y_train)
pred5 = train_model5.predict(X_test)
#print("Accuracy for model 5: %.2f" % (accuracy_score(y_test, pred5) * 100))

In [ ]:
##ISPIS RJESENJA
y_test = pred5
y_test = np.where(y_test==0,'N','Y')
student['PRIJEVREMENI_RASKID'] = y_test
student.to_csv('student.csv')

In [ ]:
#Tune Gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model6 = gsearch3.fit(X_train, y_train)
pred6 = train_model6.predict(X_test)
#print("Accuracy for model 6: %.2f" % (accuracy_score(y_test, pred6) * 100))

In [17]:
xgb2 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=4,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

train_model7 = xgb2.fit(X_train, y_train)
pred7 = train_model7.predict(X_test)
#print("Accuracy for model 7: %.2f" % (accuracy_score(y_test, pred7) * 100))

In [18]:
##ISPIS RJESENJA - pokazao se najbolji dosad
y_test = pred7
y_test = np.where(y_test==0,'N','Y')
student = pd.read_excel('eval_dataset_nan.xlsx')
student['PRIJEVREMENI_RASKID'] = y_test
student.to_csv('student.csv')